In [1]:
import os
from datetime import datetime
import traceback

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
import torchvision
import numpy as np

from tqdm import tqdm
from tensorboardX import SummaryWriter

from dataset import voc
from retinanet import model, val
from retinanet import transforms as aug


In [2]:
# consts
tag = 'debug'
split_name = 'voc-1'
root_path = '/home/voyager/data/root/voc/'

device_name = 'cuda'
batch_size = 24
depth = 50

epochs = 500
lr = 1e-5
patience = 3

image_size = 512
num_classes = 2
num_workers = 8

# info and deps
now = datetime.now()

if not os.path.exists('./result'):
    os.mkdir('./result')

result_path = './result/{}_{}_{}_{}'.format(
    tag,
    depth,
    split_name,
    now.strftime('%Y%m%d_%H%M%S')
)

summary_writer = SummaryWriter(result_path)


In [3]:
# data loader

# train

# transforms for train
train_trans = aug.Compose([
    aug.RandomCropAndPad(),
    aug.Pad(), # pad to square image
    aug.Resize(image_size, image_size),
    aug.RandomFlipLeftRight(0.5),
    aug.RandomFlipUpDown(0.5),
    aug.RandomRotate(5),
    aug.RandomTranslatePc(50, 50),
    aug.AutoLevel(min_level_rate=1, max_level_rate=1),
    aug.AutoContrast(),
    aug.RandomContrast(0.5),
    aug.Contrast(1.25),
    aug.RandomChoice([
        aug.RandomSaltPepperNoise(0.9, 0.5),
        aug.RandomSaltPepperNoise(0.95, 0.5),
        aug.RandomSaltPepperNoise(0.99, 0.5)
    ]),
    aug.ToTensor(),
    # aug.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)) # mean and std of pretrained model
])

train_set = voc.VOCDetection(
    root_path,
    image_set="{}_train".format(split_name),
    transforms=train_trans
)

train_loader = DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=voc.collate,
    num_workers=num_workers
)


In [4]:
# model, loss and optimizer

device = torch.device(device_name)

if depth == 34:
    net = model.resnet34(num_classes, pretrained=True)
elif depth == 50:
    net = model.resnet50(num_classes, pretrained=True)
elif depth == 101:
    net = model.resnet101(num_classes, pretrained=True)
elif depth == 152:
    net = model.resnet152(num_classes, pretrained=True)
    
net = net.to(device)
net = torch.nn.DataParallel(net).to(device)
net.training = True

optimizer = optim.Adam(net.parameters(), lr=lr)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    patience=patience,
    verbose=True
)

net.train()
net.module.freeze_bn()


In [ ]:
# train-val loop

for epoch in range(epochs):
    print('training epoch {}:'.format(epoch))
    
    # train
    net.train()
    net.module.freeze_bn()
    
    epoch_loss = []
    
    with tqdm(total=len(train_loader)) as pbar:
        for i, data in enumerate(train_loader):
            try:
                optimizer.zero_grad()

                # convert annos to focal loss format - [x1, y1, x2, y2, cls]
                batch_annos = []
                max_anno_count = 0

                for batch in range(len(data[1])):
                    bboxes = data[1][batch][1]
                    bboxes = bboxes.to(dtype=torch.long)
                    labels = data[1][batch][0]
                    labels = labels.to(dtype=torch.long)

                    # bboxes.shape = [4, n]
                    bboxes = torch.t(bboxes)
                    # labels.shape = [1, n]
                    labels = torch.unsqueeze(labels, 0)

                    # annos.shape = [n, 5]
                    annos = torch.cat((bboxes, labels), 0)
                    annos = torch.t(annos)

                    # record max anno count
                    anno_count = annos.shape[0]

                    if anno_count > max_anno_count:
                        max_anno_count = anno_count

                    batch_annos.append(annos)

                dummy_anno = torch.tensor([[0, 0, 0, 0, -1]])
                padded_batch_annos = []

                for anno in batch_annos:
                    if anno.shape[0] < max_anno_count:
                        dummy_count = max_anno_count - anno.shape[0]

                        for i in range(dummy_count):
                            anno = torch.cat((anno, dummy_anno), 0)

                    padded_batch_annos.append(anno)

                padded_batch_annos = torch.stack(padded_batch_annos)
                padded_batch_annos = padded_batch_annos.to(dtype=torch.float32)

                # forward
                classification_loss, regression_loss = net([
                    data[0].to(device),
                    padded_batch_annos
                ])

                classification_loss = classification_loss.mean()
                regression_loss = regression_loss.mean()

                loss = classification_loss + regression_loss
                epoch_loss.append(loss.item())

                if loss == 0:
                    continue

                loss.backward()

                torch.nn.utils.clip_grad_norm_(net.parameters(), 0.1)

                optimizer.step()

                del classification_loss
                del regression_loss
                
                pbar.update(1)
            except Exception as e:
                traceback.print_exc()
                break

    # epoch-wise work and record
    mean_loss = np.mean(epoch_loss)
    print('epoch avg loss: {}'.format(mean_loss))

    scheduler.step(mean_loss)

    # save checkpoint
    torch.save(net.module, os.path.join(result_path, '{:0>3}_{:1.4f}.pth'.format(
        epoch,
        mean_loss
    )))

    # write summary for tensorboardX
    summary_writer.add_scalar(
        '/train/loss',
        mean_loss,
        epoch
    )
    

  0%|          | 0/53 [00:00<?, ?it/s]

training epoch 0:
BoundingBoxesOnImage([BoundingBox(x1=106.0000, y1=55.0000, x2=335.0000, y2=270.0000, label=None), BoundingBox(x1=105.0000, y1=57.0000, x2=334.0000, y2=345.0000, label=None), BoundingBox(x1=190.0000, y1=120.0000, x2=420.0000, y2=316.0000, label=None), BoundingBox(x1=189.0000, y1=121.0000, x2=421.0000, y2=368.0000, label=None)], shape=(478, 624, 3))
BoundingBoxesOnImage([BoundingBox(x1=311.0000, y1=90.0000, x2=453.0000, y2=299.0000, label=None), BoundingBox(x1=259.0000, y1=1.0000, x2=456.0000, y2=304.0000, label=None)], shape=(476, 612, 3))
BoundingBoxesOnImage([BoundingBox(x1=77.0000, y1=318.0000, x2=454.0000, y2=445.0000, label=None), BoundingBox(x1=4.0000, y1=317.0000, x2=454.0000, y2=503.0000, label=None)], shape=(616, 476, 3))
BoundingBoxesOnImage([BoundingBox(x1=220.0000, y1=175.0000, x2=370.0000, y2=364.0000, label=None), BoundingBox(x1=217.0000, y1=58.0000, x2=407.0000, y2=365.0000, label=None)], shape=(478, 624, 3))
BoundingBoxesOnImage([BoundingBox(x1=292.0000

Process Process-3:
Process Process-7:
Process Process-2:
Process Process-6:
Process Process-5:
Process Process-1:
Traceback (most recent call last):
  File "/home/voyager/anaconda3/envs/pytorch_0.4.1/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Process Process-4:
Process Process-8:
Traceback (most recent call last):
  File "/home/voyager/anaconda3/envs/pytorch_0.4.1/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/voyager/anaconda3/envs/pytorch_0.4.1/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/voyager/anaconda3/envs/pytorch_0.4.1/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/voyager/anaconda3/envs/pytorch_0.4.1/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwarg

epoch avg loss: nan
training epoch 1:
BoundingBoxesOnImage([BoundingBox(x1=180.0000, y1=115.0000, x2=357.0000, y2=287.0000, label=None), BoundingBox(x1=181.0000, y1=116.0000, x2=459.0000, y2=337.0000, label=None)], shape=(612, 474, 3))
BoundingBoxesOnImage([BoundingBox(x1=94.0000, y1=305.0000, x2=299.0000, y2=461.0000, label=None), BoundingBox(x1=93.0000, y1=306.0000, x2=387.0000, y2=481.0000, label=None)], shape=(560, 420, 3))
BoundingBoxesOnImage([BoundingBox(x1=323.0000, y1=123.0000, x2=420.0000, y2=293.0000, label=None), BoundingBox(x1=323.0000, y1=124.0000, x2=422.0000, y2=411.0000, label=None)], shape=(478, 616, 3))
BoundingBoxesOnImage([BoundingBox(x1=163.0000, y1=148.0000, x2=367.0000, y2=492.0000, label=None), BoundingBox(x1=163.0000, y1=147.0000, x2=365.0000, y2=453.0000, label=None)], shape=(612, 474, 3))
BoundingBoxesOnImage([BoundingBox(x1=213.0000, y1=178.0000, x2=367.0000, y2=474.0000, label=None), BoundingBox(x1=213.0000, y1=178.0000, x2=367.0000, y2=603.0000, label=Non